# best sellers 가져오기 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time 

In [ ]:
from tqdm import tqdm

import pandas as pd
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options

chromedriver_autoinstaller.install()
options = Options()

#options.add_argument('headless')     # 크롬브라우저 숨기기
options.add_argument('window-size=1920,1080')  # 모니터 해상도 지정
options.add_argument('disable-gpu')  # 불필요한 GPU 기능을 제거해서 셀레니움 작동 속도를 올려주는 옵션
options.add_argument('--no-sandbox')
options.add_argument("--disable-dev-shm-usage")

In [ ]:
def review_cralwer(product_number):

    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9,ko;q=0.8',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        # 'Cookie': 'CheckSameSite3=IsValidSameSiteSet; AladdinUser=UID=603644110&SID=P6yC3e7Rix2KKe9hd%2fRL%2bQ%3d%3d; AladdinSession=UID=603644110&SID=P6yC3e7Rix2KKe9hd%2fRL%2bQ%3d%3d; AladdinUS=zExY69KPCoN91xVRs4hFdg%3d%3d&USA=0; _gcl_au=1.1.1070067853.1683997944; _BS_GUUID=3g7FfVVdczkjLLHpC1bD2LC2PshbpPPt5W8lS4y4; _hackle_hid=4fa31a98-e2d8-4841-bca0-8c148526122f; _ga=GA1.1.1510244557.1683997944; _hackle_session_id_LFF0kdW8JPQCyXe70FlpBaud=1683997944678.081a8803; _TRK_AUIDA_13987=3e92c7c2893c068100a3402f000fc079:2; _TRK_ASID_13987=df8a436d375e9c6807c0ebb68d7d0070; refererURL=https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=309295168; _hackle_last_event_ts_LFF0kdW8JPQCyXe70FlpBaud=1684037934522; _ga_VKYKBC0ZHH=GS1.1.1684037888.2.1.1684037994.47.0.0',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Referer': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=309295168',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }

    params = {
        'ProductItemId': product_number,
        'itemId': product_number,
        'pageCount': '100', # 몇개 리뷰 가져 올건지 
        'communitytype': 'CommentReview',
        'nemoType': '-1',
        'page': '1',
        'startNumber': '1',
        'endNumber': '100', # 마지막 번호
        'sort': '2',
        'IsOrderer': '2',
        'BranchType': '1',
        'IsAjax': 'true',
        'pageType': '0',
    }

    response = requests.get(
        'https://www.aladin.co.kr/ucl/shop/product/ajax/GetCommunityListAjax.aspx',
        params=params,
        headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all div elements with id starting with "spnPaper"
    target_elements = soup.find_all('div', id=lambda value: value and value.startswith('spnPaper'))
    # Extract the content of the elements
    review_list = [element.get_text(strip=True) for element in target_elements]
    review_list = review_list[1::2] # 스포일러 머시깽이 버려야 해서 
    
    return review_list

In [4]:
def description_crawler(contents_id):
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9,ko;q=0.8',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        # 'Cookie': 'CheckSameSite3=IsValidSameSiteSet; AladdinUser=UID=603644110&SID=P6yC3e7Rix2KKe9hd%2fRL%2bQ%3d%3d; AladdinSession=UID=603644110&SID=P6yC3e7Rix2KKe9hd%2fRL%2bQ%3d%3d; AladdinUS=zExY69KPCoN91xVRs4hFdg%3d%3d&USA=0; _gcl_au=1.1.1070067853.1683997944; _BS_GUUID=3g7FfVVdczkjLLHpC1bD2LC2PshbpPPt5W8lS4y4; _hackle_hid=4fa31a98-e2d8-4841-bca0-8c148526122f; _ga=GA1.1.1510244557.1683997944; _hackle_session_id_LFF0kdW8JPQCyXe70FlpBaud=1683997944678.081a8803; _TRK_AUIDA_13987=3e92c7c2893c068100a3402f000fc079:2; _TRK_ASID_13987=df8a436d375e9c6807c0ebb68d7d0070; ckMobile=SiteMode=Mobile; divGoodsEventBottomLayerCount=1; refererURL=https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=309295168; _ga_VKYKBC0ZHH=GS1.1.1684037888.2.1.1684044602.60.0.0; _hackle_last_event_ts_LFF0kdW8JPQCyXe70FlpBaud=1684044603007',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Referer': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=309295168',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }

    params = {
        'ISBN': contents_id,
        'name': 'Introduce',
        'type': '0',
        'date': '15',
    }

    response = requests.get('https://www.aladin.co.kr/shop/product/getContents.aspx', params=params, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    element = soup.find(class_='Ere_prod_mconts_R')
    contents = element.get_text(strip='text')
    return contents

In [5]:
# 책 100개 목록 가져오기 : 타이틀, url만

In [52]:
book_df = pd.DataFrame()

for page in [1,2,3]:
    
    # 한 페이지에 50개씩 2페이지 가져와야합니다
    response = requests.get(f'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page={page}&cnt=50&SortOrder=1')
    soup = BeautifulSoup(response.content, 'html.parser')
    product_url_element = soup.select('.ss_book_list li a.bo3')
    
    product_name = [i.get_text() for i in product_url_element]
    product_url = [i['href'] for i in product_url_element]
    
    tmp_df = pd.DataFrame()
    tmp_df['name'] = product_name
    tmp_df['url'] = product_url
    
    book_df = pd.concat([book_df, tmp_df])

In [53]:
# 상세 정보 및 리뷰 데이터 가져 오기 

In [8]:
driver = webdriver.Chrome(options=options)

In [9]:
data_list = []

for url in tqdm(book_df['url']):
    try:
        driver.get(url)
        driver.implicitly_wait(1)  # 10초간 대기

        author = driver.find_element(By.XPATH, '//*[@id="Ere_prod_allwrap"]/div[3]/div[2]/div[1]/div/ul/li[3]/a[1]')
        author = author.text

        price = driver.find_element(By.XPATH, '//*[@id="Ere_prod_allwrap"]/div[4]/div[4]/div/div[3]/ul/li[1]/div[2]')
        price = price.text

        reivew_count = driver.find_element(By.XPATH, '//*[@id="wa_product_top1_wa_Top_Ranking_pnlRanking"]/div[2]/a[3]')
        reivew_count = reivew_count.text

        # 리뷰 가져 오기 
        product_number = url.split('ItemId=')[1] 
        review_list = review_cralwer(product_number)

        data_list.append([author, price, reivew_count, review_list])
    
    except:
        data_list.append(['', '', '', ''])

100%|█████████████████████████████████████████| 150/150 [15:46<00:00,  6.31s/it]


In [10]:
import pandas as pd

In [11]:
contents_id = pd.read_csv('./Downloads/알라딘_주간+베스트_국내도서_2023년5월2주_20230514.xls')

In [12]:
contents_id = contents_id.iloc[:150]['ISBN']

In [13]:
contents_list =[]
for i in tqdm(contents_id):
    try:
        contents_list.append(description_crawler(i))
    except:
        contents_list.append('')

100%|█████████████████████████████████████████| 150/150 [00:22<00:00,  6.79it/s]


In [70]:
second_df = pd.DataFrame(data_list, columns=['저자', '가격', '리뷰수', '리뷰내용'])

In [79]:
second_df['책소개'] = contents_list

In [72]:
# 컬럼 수정
second_df['리뷰수'] = second_df['리뷰수'].str.replace('100자평', '')

In [73]:
second_df['리뷰수'] = second_df['리뷰수'].apply(lambda x: re.sub(r'\(|\)', '', x))


In [74]:
second_df['가격'] = second_df['가격'].str.replace('원', '')

In [75]:
second_df['제목'] = book_df['name'].to_list()

In [76]:
second_df['사이트'] = '알라딘'

In [77]:
second_df = second_df.reset_index()

In [78]:
second_df['순위'] = second_df['index'] + 1

In [81]:
second_df = second_df[['사이트', '순위', '제목', '저자', '가격', '책소개', '리뷰수', '리뷰내용']]

In [82]:
second_df.to_excel('알라딘.xlsx', index=False)